# Xây dựng mô hình RNN cho bài toán Nhận dạng thực thể tên (NER)

## Task 1: Tải và tiền xử lý dữ liệu

In [55]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [56]:
# Trích xuất câu và nhãn dạng số
train_sentences = dataset["train"]["tokens"]
train_tags = dataset["train"]["ner_tags"]

val_sentences = dataset["validation"]["tokens"]
val_tags = dataset["validation"]["ner_tags"]

test_sentences = dataset["test"]["tokens"]
test_tags = dataset["test"]["ner_tags"]

# Lấy ánh xạ số -> tên nhãn
label_names = dataset["train"].features["ner_tags"].feature.names
print("Danh sách nhãn:", label_names)

# Hàm chuyển từng nhãn số sang nhãn text
def convert_tags(tag_ids):
    return [label_names[i] for i in tag_ids]

# Chuyển đổi toàn bộ train/val/test
train_tags_str = [convert_tags(seq) for seq in train_tags]
val_tags_str = [convert_tags(seq) for seq in val_tags]
test_tags_str = [convert_tags(seq) for seq in test_tags]

# In thử một câu và nhãn
print("Sentence:", train_sentences[0])
print("NER IDs:", train_tags[0])
print("NER Labels:", train_tags_str[0])


Danh sách nhãn: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
Sentence: ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
NER IDs: [3, 0, 7, 0, 0, 0, 7, 0, 0]
NER Labels: ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [57]:
word_to_ix = {
    "<PAD>": 0,
    "<UNK>": 1
}

# Duyệt toàn bộ câu để thêm từ vào từ điển
for sentence in train_sentences:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {}
for tag in label_names:
    if tag not in tag_to_ix:
        tag_to_ix[tag] = len(tag_to_ix)

print("Số lượng từ trong word_to_ix:", len(word_to_ix))
print("Số lượng nhãn trong tag_to_ix:", len(tag_to_ix))

# In thử vài phần tử
print("Ví dụ trong word_to_ix:", list(word_to_ix.items())[:10])
print("tag_to_ix:", tag_to_ix)


Số lượng từ trong word_to_ix: 23625
Số lượng nhãn trong tag_to_ix: 9
Ví dụ trong word_to_ix: [('<PAD>', 0), ('<UNK>', 1), ('EU', 2), ('rejects', 3), ('German', 4), ('call', 5), ('to', 6), ('boycott', 7), ('British', 8), ('lamb', 9)]
tag_to_ix: {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}


## Task 2: Tạo PyTorch Dataset và DataLoader

In [58]:
import torch
from torch.utils.data import Dataset, DataLoader

class NERDataset(Dataset):
    def __init__(self, sentences, tags, word_to_ix, tag_to_ix):
        """
        sentences: list[list[token]]
        tags: list[list[label_str]]
        """
        self.sentences = sentences
        self.tags = tags
        self.word_to_ix = word_to_ix
        self.tag_to_ix = tag_to_ix

        # id đặc biệt
        self.unk_id = word_to_ix["<UNK>"]

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        tokens = self.sentences[index]
        tag_seq = self.tags[index]

        # Chuyển từ sang chỉ số
        sent_idx = [
            self.word_to_ix.get(tok, self.unk_id)
            for tok in tokens
        ]

        # Chuyển nhãn sang chỉ số
        tag_idx = [
            self.tag_to_ix[tag] for tag in tag_seq
        ]

        return (
            torch.tensor(sent_idx, dtype=torch.long),
            torch.tensor(tag_idx, dtype=torch.long)
        )




In [59]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """
    batch: list of (sent_tensor, tag_tensor)
    """
    sentences, tags = zip(*batch)

    # Padding cho câu
    pad_id = word_to_ix["<PAD>"]  # index của <PAD>
    sentences_padded = pad_sequence(sentences, batch_first=True, padding_value=pad_id)

    # Padding cho nhãn
    tags_padded = pad_sequence(tags, batch_first=True, padding_value=-1)

    return sentences_padded, tags_padded

train_dataset = NERDataset(
    train_sentences, train_tags_str, word_to_ix, tag_to_ix
)

val_dataset = NERDataset(
    val_sentences, val_tags_str, word_to_ix, tag_to_ix
)

test_dataset = NERDataset(
    test_sentences, test_tags_str, word_to_ix, tag_to_ix
)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=collate_fn
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=collate_fn
)


## Task 3: Xây dựng Mô hình RNN

In [60]:
import torch
import torch.nn as nn

class LSTMForNER(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes, padding_idx=0, bidirectional=True):
        """
        vocab_size: số từ trong từ điển
        embedding_dim: kích thước embedding
        hidden_dim: số neuron ẩn của LSTM
        num_classes: số lượng nhãn NER
        padding_idx: chỉ số dùng để pad
        bidirectional: True nếu dùng BiLSTM
        """
        super(LSTMForNER, self).__init__()

        # 1. Embedding layer
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=padding_idx
        )

        # 2. LSTM layer
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=bidirectional
        )

        # 3. Linear layer: từ hidden_dim*2 nếu BiLSTM → num_classes
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), num_classes)

    def forward(self, x, lengths=None):
        """
        x: (batch_size, seq_len) chứa index từ
        lengths: độ dài thực của từng câu (dùng cho packed sequence)
        """
        # Embedding
        embedded = self.embedding(x)  # (batch_size, seq_len, embedding_dim)

        if lengths is not None:
            packed_embedded = nn.utils.rnn.pack_padded_sequence(
                embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
            )
            packed_output, _ = self.lstm(packed_embedded)
            output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        else:
            output, _ = self.lstm(embedded)  # (batch_size, seq_len, hidden_dim*2 nếu bidirectional)

        logits = self.fc(output)  # (batch_size, seq_len, num_classes)
        return logits


## Task 4: Huấn luyện Mô hình

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim

# Thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Giả sử bạn đã có:
# model, train_loader, val_loader, tag_to_ix
padding_tag_id = -1  # giống giá trị padding nhãn trong collate_fn

# Khởi tạo mô hình
vocab_size = len(word_to_ix)
embedding_dim = 100
hidden_dim = 128
output_size = len(tag_to_ix)
padding_idx = word_to_ix["<PAD>"]

model = LSTMForNER(vocab_size, embedding_dim, hidden_dim, output_size, padding_idx=padding_idx)
model.to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Loss function
criterion = nn.CrossEntropyLoss(ignore_index=padding_tag_id)


In [62]:
num_epochs = 5  # hoặc 5 tùy nhu cầu

for epoch in range(1, num_epochs+1):
    model.train()
    total_loss = 0.0
    num_batches = 0

    for batch_sent, batch_tag in train_loader:
        batch_sent = batch_sent.to(device)  # (batch_size, seq_len)
        batch_tag = batch_tag.to(device)    # (batch_size, seq_len)

        # 1. Xóa gradient cũ
        optimizer.zero_grad()

        # 2. Forward pass
        logits = model(batch_sent)  # (batch_size, seq_len, num_classes)

        # 3. Tính loss
        # reshape logits và labels để CrossEntropyLoss nhận dạng
        # từ (batch, seq_len, num_classes) → (batch*seq_len, num_classes)
        logits_flat = logits.view(-1, output_size)
        tags_flat = batch_tag.view(-1)

        loss = criterion(logits_flat, tags_flat)

        # 4. Backward pass
        loss.backward()

        # 5. Cập nhật trọng số
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

    avg_loss = total_loss / num_batches
    print(f"Epoch {epoch}/{num_epochs} - Loss trung bình: {avg_loss:.4f}")


Epoch 1/5 - Loss trung bình: 0.5765
Epoch 2/5 - Loss trung bình: 0.2659
Epoch 3/5 - Loss trung bình: 0.1529
Epoch 4/5 - Loss trung bình: 0.0886
Epoch 5/5 - Loss trung bình: 0.0489


In [63]:
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

def evaluate(model, data_loader, tag_to_ix, device):
    model.eval()
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}  # ánh xạ chỉ số → tên nhãn

    total_tokens = 0
    correct_tokens = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_sent, batch_tag in data_loader:
            batch_sent = batch_sent.to(device)
            batch_tag = batch_tag.to(device)

            logits = model(batch_sent)  # (batch, seq_len, num_classes)
            preds = torch.argmax(logits, dim=-1)  # (batch, seq_len)

            # Tính accuracy bỏ qua padding
            mask = batch_tag != -1
            total_tokens += mask.sum().item()
            correct_tokens += ((preds == batch_tag) & mask).sum().item()

            # Chuẩn bị dữ liệu cho seqeval
            preds_list = []
            labels_list = []

            for p, t, m in zip(preds.cpu(), batch_tag.cpu(), mask.cpu()):
                p_seq = [ix_to_tag[ix.item()] for ix, mask_val in zip(p, m) if mask_val]
                t_seq = [ix_to_tag[ix.item()] for ix, mask_val in zip(t, m) if mask_val]
                preds_list.append(p_seq)
                labels_list.append(t_seq)

            all_preds.extend(preds_list)
            all_labels.extend(labels_list)

    accuracy = correct_tokens / total_tokens
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)

    print(f"Accuracy (token-level): {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
    print("\nBáo cáo chi tiết theo thực thể:")
    print(classification_report(all_labels, all_preds))

    return accuracy, precision, recall, f1


In [64]:
print(f"="*30, "Trên tập Test", "="*30)
accuracy, precision, recall, f1 = evaluate(model, test_loader, tag_to_ix, device)
print(f"="*30, "Trên tập Val", "="*30)
accuracy, precision, recall, f1 = evaluate(model, val_loader, tag_to_ix, device)

print(f"Độ chính xác trên tập validation: {accuracy:.4f}")


============================== Trên tập Test ==============================
Accuracy (token-level): 0.9188
Precision: 0.7216, Recall: 0.5351, F1-score: 0.6145

Báo cáo chi tiết theo thực thể:
              precision    recall  f1-score   support

         LOC       0.85      0.67      0.75      1668
        MISC       0.61      0.58      0.60       702
         ORG       0.69      0.51      0.59      1661
         PER       0.66      0.40      0.50      1617

   micro avg       0.72      0.54      0.61      5648
   macro avg       0.70      0.54      0.61      5648
weighted avg       0.72      0.54      0.61      5648

============================== Trên tập Val ==============================
Accuracy (token-level): 0.9438
Precision: 0.8213, Recall: 0.6653, F1-score: 0.7351

Báo cáo chi tiết theo thực thể:
              precision    recall  f1-score   support

         LOC       0.90      0.76      0.83      1837
        MISC       0.79      0.71      0.75       922
         ORG       

In [65]:
def predict_sentence(sentence, model, word_to_ix, ix_to_tag, device):
    """
    sentence: str, câu mới
    model: mô hình NER đã huấn luyện
    word_to_ix: từ điển từ -> index
    ix_to_tag: từ điển index -> nhãn string
    """
    model.eval()

    # Tách câu thành danh sách token (simple split, có thể dùng tokenizer nâng cao)
    tokens = sentence.split()

    # Chuyển token -> index
    indices = [word_to_ix.get(tok, word_to_ix["<UNK>"]) for tok in tokens]

    # Tạo tensor và đưa lên device
    input_tensor = torch.tensor([indices], dtype=torch.long).to(device)

    with torch.no_grad():
        logits = model(input_tensor)  # (1, seq_len, num_classes)
        pred_indices = torch.argmax(logits, dim=-1).squeeze(0)  # (seq_len,)

    # Chuyển chỉ số sang nhãn
    pred_tags = [ix_to_tag[ix.item()] for ix in pred_indices]

    # In ra cặp (từ, nhãn)
    for tok, tag in zip(tokens, pred_tags):
        print(f"{tok}\t{tag}")

    return list(zip(tokens, pred_tags))


In [66]:
# Chuẩn bị ánh xạ ngược
ix_to_tag = {v:k for k,v in tag_to_ix.items()}

# Ví dụ dự đoán
sentence = "VNU University is located in Hanoi"
pred_pairs = predict_sentence(sentence, model, word_to_ix, ix_to_tag, device)


VNU	B-ORG
University	I-ORG
is	O
located	O
in	O
Hanoi	B-LOC
